In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
import torch.nn.functional as F 

from torchvision import transforms

from torch.utils.data import TensorDataset, DataLoader

In [84]:
torch.manual_seed(0)

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
path = "drive/MyDrive/Colab Notebooks/fashion-mnist.csv"
df = pd.read_csv(path)

In [73]:
X = torch.div(torch.Tensor(np.array(df.iloc[:, 1:])),255).type(torch.FloatTensor)
y = torch.Tensor(np.array(df.iloc[:, 0:1].values)).reshape(-1).type(torch.LongTensor)

In [74]:
batch_size = 64

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test,y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [75]:
class FashionMNISTNN(nn.Module):
    def __init__(self, input_dim, h1,h2, output_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, output_dim)

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):                           
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.softmax(self.fc3(x),dim=1)                  

        return x

In [63]:
def calc_accuracy(a, y):                            
    a = torch.max(a, dim=1)[1]
    assert not torch.any(torch.isnan(a))
    return torch.mean((a == y).float()).item()

In [80]:
model = FashionMNISTNN(784,128,32,10)

optimizer = optim.Adam(model.parameters(), lr=float(0.01))

In [85]:
num_epochs = 25
loss_history = []
accuracy_history = []

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for image, label in train_loader:
        z = model(image)
        loss = F.cross_entropy(z, label)
        accuracy = calc_accuracy(z, label)

        optimizer.zero_grad()      
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() / len(train_dataset) * batch_size
        epoch_accuracy += accuracy / len(train_dataset) * batch_size
    
    loss_history.append(epoch_loss)
    accuracy_history.append(epoch_accuracy)

    if epoch % 5 == 0:
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, round(epoch_loss, 3), round(100*epoch_accuracy, 3)))

Epoch: 0, Loss: 1.695, Accuracy: 76.6
Epoch: 5, Loss: 1.759, Accuracy: 70.2
Epoch: 10, Loss: 1.72, Accuracy: 74.15
Epoch: 15, Loss: 1.792, Accuracy: 66.938
Epoch: 20, Loss: 1.708, Accuracy: 75.362


In [86]:
num_epochs = 5
avg_loss = 0
avg_accuracy = 0

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for image, label in test_loader:
        z = model(image.float())
        loss = F.cross_entropy(z, label)
        accuracy = calc_accuracy(z, label)

        epoch_loss += loss.item() / len(test_dataset) * batch_size
        epoch_accuracy += accuracy / len(test_dataset) * batch_size

    avg_loss += epoch_loss / num_epochs
    avg_accuracy += epoch_accuracy / num_epochs

print("Number of Epochs tested: {}, Average loss: {}, Average Accuracy: {}".format(num_epochs, round(avg_loss, 3), round(100*avg_accuracy, 3)))

Number of Epochs tested: 5, Average loss: 1.777, Average Accuracy: 74.36
